# DESCRIPTION
#**Stock Trading Using Deep Q-Learning**

## **Problem Statement**

Prepare an agent by implementing Deep Q-Learning that can perform unsupervised trading in stock trade. The aim of this project is to train an agent that uses Q-learning and neural networks to predict the profit or loss by building a model and implementing it on a dataset that is available for evaluation.


The stock trading index environment provides the agent with a set of actions:<br>
* Buy<br>
* Sell<br>
* Sit

This project has following sections:
* Import libraries 
* Create a DQN agent
* Preprocess the data
* Train and build the model
* Evaluate the model and agent
<br><br>

**Steps to perform**<br>

In the section **create a DQN agent**, create a class called agent where:
* Action size is defined as 3
* Experience replay memory to deque is 1000
* Empty list for stocks that has already been bought
* The agent must possess the following hyperparameters:<br>
  * gamma= 0.95<br>
  * epsilon = 1.0<br>
  * epsilon_final = 0.01<br>
  * epsilon_decay = 0.995<br>


    Note: It is advised to compare the results using different values in hyperparameters.

* Neural network has 3 hidden layers
* Action and experience replay are defined

By Edson Teixeira
teixeiraedson252@gmail.com
January 4th 2022



## **Solution**

### **Import the libraries** 

In [2]:

#import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# import keras
# from keras.models import Sequential
# from keras.models import load_model
# from keras.layers import Dense
# from keras.optimizers import Adam
import numpy as np
import random
from collections import deque

### **Create a DQN agent**

**Use the instruction below to prepare an agent**


In [3]:
# Action space include 3 actions: Buy, Sell, and Sit
#Setting up the experience replay memory to deque with 1000 elements inside it
#Empty list with inventory is created that contains the stocks that were already bought
#Setting up gamma to 0.95, that helps to maximize the current reward over the long-term
#Epsilon parameter determines whether to use a random action or to use the model for the action. 
#In the beginning random actions are encouraged, hence epsilon is set up to 1.0 when the model is not trained.
#And over time the epsilon is reduced to 0.01 in order to decrease the random actions and use the trained model
#We're then set the speed of decreasing epsililon in the epsilon_decay parameter

#Defining our neural network:
#Define the neural network function called _model and it just takes the keyword self
#Define the model with Sequential()
#Define states i.e. the previous n days and stock prices of the days
#Defining 3 hidden layers in this network
#Changing the activation function to relu because mean-squared error is used for the loss

#Import Neural Network Packages
import numpy as np
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Flatten, Conv2D
#from tensorflow.keras.optimizers import Adam




In [4]:
class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = load_model(model_name) if is_eval else self._model()
    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))
        return model
    def act(self, state):
        if not self.is_eval and random.random()<= self.epsilon:
            return random.randrange(self.action_size)
        options = self.model.predict(state)
        return np.argmax(options[0])
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [5]:
# For a given state and action, reward ---> Q learning
# Experience Replay says you can store the above so that agent can take decision not only on the basis of
# Q table but also on the basis of deque !

### **Preprocess the stock market data**

In [6]:
import math

# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
	vec = []
	lines = open("" + key + ".csv", "r").read().splitlines()

	for line in lines[1:]:
		vec.append(float(line.split(",")[4]))

	return vec

# returns the sigmoid
def sigmoid(x):
	return 1 / (1 + math.exp(-x))

# returns an an n-day state representation ending at time t
def getState(data, t, n):
	d = t - n + 1
	block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
	res = []
	for i in range(n - 1):
		res.append(sigmoid(block[i + 1] - block[i]))

	return np.array([res])


### **Train and build the model**

In [ ]:
import sys

if len(sys.argv) != 4:
	print ("Usage: python train.py [stock] [window] [episodes]")
	exit()


stock_name = input("Enter stock_name, window_size, Episode_count")
#Fill the given information when prompted: 
#Enter stock_name = GSPC_Training_Dataset
#window_size = 10
#Episode_count = 100 or it can be 10 or 20 or 30 and so on.

window_size = input()
episode_count = input()
stock_name = str(stock_name)
window_size = int(window_size)
episode_count = int(episode_count)

agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 2460 #Here we have reduced the data points please do this as well.
batch_size = 32

for e in range(episode_count + 1):
	#print ("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			#print ("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			#print ("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print ("--------------------------------")
			print ("Total Profit: " + formatPrice(total_profit))
			

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	#if e % 10 == 0:
		agent.model.save("model_ep" + str(e))

Usage: python train.py [stock] [window] [episodes]


Enter stock_name, window_size, Episode_count GSPC_Training_Dataset
 5
 10


[2021-08-24 20:11:20.858 tensorflow-2-3-gpu--ml-g4dn-xlarge-c85184389676cdfa7bdf06745c9b:42 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-24 20:11:20.883 tensorflow-2-3-gpu--ml-g4dn-xlarge-c85184389676cdfa7bdf06745c9b:42 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

INFO:tensorflow:Assets written to: model_ep8/assets


INFO:tensorflow:Assets written to: model_ep8/assets


INFO:tensorflow:Assets written to: model_ep8/assets


INFO:tensorflow:Assets written to: model_ep8/assets


INFO:tensorflow:Assets written to: model_ep8/assets


### **Evaluate the model and agent**

In [7]:
import sys
from tensorflow.keras.models import load_model


if len(sys.argv) != 3:
	print ("Usage: python evaluate.py [stock] [model]")
	exit()


stock_name = input("Enter Stock_name, Model_name")
model_name = input()
#Note: 
#Fill the given information when prompted: 
#Enter stock_name = GSPC_Evaluation_Dataset
#Model_name = respective model name

model = load_model("" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []

for t in range(l):
	action = agent.act(state)

	# sit
	next_state = getState(data, t + 1, window_size + 1)
	reward = 0

	if action == 1: # buy
		agent.inventory.append(data[t])
		print ("Buy: " + formatPrice(data[t]))

	elif action == 2 and len(agent.inventory) > 0: # sell
		bought_price = agent.inventory.pop(0)
		reward = max(data[t] - bought_price, 0)
		total_profit += data[t] - bought_price
		print ("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

	done = True if t == l - 1 else False
	agent.memory.append((state, action, reward, next_state, done))
	state = next_state

	if done:
		print ("--------------------------------")
		print (stock_name + " Total Profit: " + formatPrice(total_profit))



Enter Stock_name, Model_name GSPC_Evaluation_Dataset
 model_ep9


Buy: $1276.56
Buy: $1273.85
Buy: $1269.75
Sell: $1274.48 | Profit: -$2.08
Sell: $1285.96 | Profit: $12.11
Sell: $1283.76 | Profit: $14.01
Buy: $1293.24
Buy: $1295.02
Buy: $1280.26
Buy: $1283.35
Sell: $1290.84 | Profit: -$2.40
Sell: $1291.18 | Profit: -$3.84
Buy: $1296.63
Buy: $1299.54
Sell: $1276.34 | Profit: -$3.92
Buy: $1286.12
Buy: $1307.59
Buy: $1307.10
Buy: $1310.87
Sell: $1319.05 | Profit: $35.70
Buy: $1324.57
Sell: $1320.88 | Profit: $24.25
Buy: $1321.87
Buy: $1329.15
Sell: $1332.32 | Profit: $32.78
Buy: $1328.01
Buy: $1336.32
Buy: $1340.43
Sell: $1343.01 | Profit: $56.89
Buy: $1315.44
Sell: $1307.40 | Profit: -$0.19
Buy: $1306.10
Sell: $1319.88 | Profit: $12.78
Sell: $1327.22 | Profit: $16.35
Sell: $1306.33 | Profit: -$18.24
Buy: $1308.44
Buy: $1330.97
Buy: $1310.13
Buy: $1321.82
Sell: $1320.02 | Profit: -$1.85
Buy: $1304.28
Sell: $1296.39 | Profit: -$32.76
Buy: $1256.88
Sell: $1273.72 | Profit: -$54.29
Buy: $1279.21
Sell: $1298.38 | Profit: -$37.94
Buy: $1293.77
Buy: $1297.54


**Note: Run the training section for considerable episodes so that while evaluating the model it can generate significant profit.** 
